In [ ]:
要创建套接字，必须使用 socket.socket()函数，它一般的语法如下。
socket(socket_family, socket_type, protocol=0)
其中，socket_family 是 AF_UNIX 或 AF_INET（如前所述），socket_type 是 SOCK_STREAM
或 SOCK_DGRAM（也如前所述）。protocol 通常省略，默认为 0。

In [ ]:
所以，为了创建 TCP/IP 套接字，可以用下面的方式调用 socket.socket()。
tcpSock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [ ]:
同样，为了创建 UDP/IP 套接字，需要执行以下语句。
udpSock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [ ]:
因为有很多 socket 模块属性，所以此时使用“from module import *”这种导入方式可以
接受，不过这只是其中的一个例外。如果使用“from socket import *”，那么我们就把 socket
属性引入到了命名空间中。虽然这看起来有些麻烦，但是通过这种方式将能够大大缩短代码，
正如下面所示。
tcpSock = socket(AF_INET, SOCK_STREAM)
一旦有了一个套接字对象，那么使用套接字对象的方法将可以进行进一步的交互

In [ ]:
表 2-1 列出了最常见的套接字方法。在下一节中，我们将使用其中的一些方法创建 TCP
和 UDP 客户端与服务器。虽然我们专注于网络套接字，但这些方法与使用本地/不联网的套
接字时有类似的含义。
表 2-1 常见的套接字对象方法和属性
名 称 描 述 
服务器套接字方法
s.bind() 将地址（主机名、端口号对）绑定到套接字上
s.listen() 设置并启动 TCP 监听器
s.accept() 被动接受 TCP 客户端连接，一直等待直到连接到达（阻塞）
客户端套接字方法
s.connect() 主动发起 TCP 服务器连接
s.connect_ex() connect()的扩展版本，此时会以错误码的形式返回问题，而不是抛出一个异常
普通的套接字方法
s.recv() 接收 TCP 消息
s.recv_into()①
接收 TCP 消息到指定的缓冲区
50 第 1 部分 通用应用主题
（续表） 
名 字 描 述 
s.send() 发送 TCP 消息
s.sendall() 完整地发送 TCP 消息
s.recvfrom() 接收 UDP 消息
s.recvfrom_into()①
接收 UDP 消息到指定的缓冲区
s.sendto() 发送 UDP 消息
s.getpeername() 连接到套接字（TCP）的远程地址
s.getsockname() 当前套接字的地址
s.getsockopt() 返回给定套接字选项的值
s.setsockopt() 设置给定套接字选项的值
s.shutdown() 关闭连接
s.close() 关闭套接字
s.detach()② 在未关闭文件描述符的情况下关闭套接字，返回文件描述符
s.ioctl()③
控制套接字的模式（仅支持 Windows）
面向阻塞的套接字方法
s.setblocking() 设置套接字的阻塞或非阻塞模式
s.settimeout()④ 设置阻塞套接字操作的超时时间
s.gettimeout()④ 获取阻塞套接字操作的超时时间
面向文件的套接字方法
s.fileno() 套接字的文件描述符
s.makefile() 创建与套接字关联的文件对象
数据属性
s.family① 套接字家族
s.type①
套接字类型
s.proto①
套接字协议
① Python 2.5 中新增。
② Python 3.2 中新增。
③ Python 2.6 中新增，仅仅支持 Windows 平台；POSIX 系统可以使用 functl 模块函数。
④ Python 2.3 中新增

In [ ]:
核心提示：在不同的计算机上分别安装客户端和服务器来运行网络应用程序
在本章众多的例子中，你会经常看到指示主机“localhost”的代码和输出，或者看到
127.0.0.1 的 IP 地址。在这里的示例中，客户端和服务器运行在同一台计算机上。不过，
鼓励读者修改主机名，并将代码复制到不同的计算机上，因为这样开发的代码运行起来更
加有趣，让计算机通过网络相互通信，然后可以看到网络程序确实能够工作！


In [ ]:
创建 TCP 服务器

In [ ]:
首先，我们将展现创建通用 TCP 服务器的一般伪代码，然后对这些代码的含义进行一般
性的描述。需要记住的是，这仅仅是设计服务器的一种方式。一旦熟悉了服务器设计，那么
你将能够按照自己的要求修改下面的伪代码来操作服务器。
ss = socket() # 创建服务器套接字
ss.bind() # 套接字与地址绑定
ss.listen() # 监听连接
inf_loop: # 服务器无限循环
    cs = ss.accept() # 接受客户端连接
    comm_loop: # 通信循环
        cs.recv()/cs.send() # 对话（接收/发送）
    cs.close() # 关闭客户端套接字
ss.close() # 关闭服务器套接字#（可选）

In [ ]:
核心提示：多线程处理客户端请求
我们没在该例子中实现这一点，但将一个客户端请求切换到一个新线程或进程来完成
客户端处理也是相当普遍的。SocketServer 模块是一个以 socket 为基础而创建的高级套接
字通信模块，它支持客户端请求的线程和多进程处理。可以参考文档或在第 4 章的练习部
分获取 SocketServer 模块的更多信息

In [7]:
# TCP时间戳服务器
from socket import *
from time import ctime
HOST = ''
PORT = 21567
BUFSIZ = 1024
ADDR = (HOST,PORT)

tcpSerSock = socket(AF_INET,SOCK_STREAM)
tcpSerSock.bind(ADDR)
tcpSerSock.listen(5)

try:
    while True:
        print('waiting for connection...')
        tcpCliSock, addr = tcpSerSock.accept()
        print ('...connected from:',addr)

        while True:
            data = tcpCliSock.recv(BUFSIZ)
            print(type(data),data)
            if not data:
                break
    #         shuju='%s %s'%(ctime(),data).encode()
            a='[%s] %s'%(bytes(ctime(),'utf-8'),data)
            a=a.encode()
            tcpCliSock.send(a)


            tcpCliSock.close()
except:  
    tcpSerSock.close()
    
        

waiting for connection...
...connected from: ('127.0.0.1', 1616)
<class 'bytes'> b'hi'


In [5]:
# 创建 TCP 客户端


[Mon Feb 14 10:11:59 2022] hi


In [ ]:
创建客户端比服务器要简单得多。与对 TCP 服务器的描述类似，本节将先给出附带解释
的伪代码，然后揭示真相。
cs = socket() # 创建客户端套接字
cs.connect() # 尝试连接服务器
comm_loop: # 通信循环
cs.send()/cs.recv() # 对话（发送/接收）
cs.close() # 关闭客户端套接字

In [ ]:
# TCP 时间戳客户端
from socket import *

HOST = 'localhost'
PORT = 21567
BUFSIZ = 1024
ADDR = (HOST,PORT)

tcpCliSock = socket(AF_INET,SOCK_STREAM)
tcpCliSock.connect(ADDR)

while True:
    data = raw_input('> ')
    if not data:
        break
    tcpCliSock.send(data)
    data = tcpCliSock.recv(BUFSIZ)
    if not data:
        break
    print (data)
tcpCliSock.close()





NameError: name 'raw_input' is not defined